In [8]:
import requests
import json
import overpy as opy
import pandas as pd
import geopandas as gpd
import osm2geojson
import sys
import matplotlib.pyplot as plt
sys.path.append('..')
import utils #import utils from the directory up above

In [2]:
overpass_query = """
    [bbox:44.453388800301774,-0.56304931640625,46.240651955001695,2.3345947265625]
    [timeout:25]
    ;
    (
    node["landuse"="vineyard"];
    way["landuse"="vineyard"];
    relation["landuse"="vineyard"];
    );
    out center;
    """

In [3]:
france_vineyards = utils.OSMGeometries(overpass_query)

Call the function to get all of the ways nodes

In [22]:
france_ways = france_vineyards.ways_dataframe()
france_ways

I saved the above dataframe into a csv and will load it in. I will then construct geometries from these lons/lats and combine them into one dataframe

In [10]:
france_way_csv = pd.read_csv(r'C:\Users\brunolopez\mldata\osm\vineyards_ways.csv')
france_way_shp = utils.create_ways_geometry(france_way_csv) #from the utils package 

The next part of this analysis will require the geemap package, which requires a Google Earth Engine account. This will allow us to use Sentinel and Landsat 
data as well as other datasets that google earth engine has

In [10]:
import geemap
import ee

#ee.Authenticate() uncomment if first time running script.
ee.Initialize()

In [11]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Let's load in the polygons from the earlier analysis from before.

In [17]:
france_vineyards = ee.FeatureCollection('projects/ee-brunohlope/assets/vineyards_polygons')
Map.addLayer(france_vineyards, {} , 'Vineyards')
Map.centerObject(france_vineyards) #View where the map is 
Map

Map(bottom=617.0, center=[-39.368279149160124, 28.125000000000004], controls=(WidgetControl(options=['position…

## The Fun

Let's grab some Sentinel-2 Imagery for these polygons and overlay them with our image.